# Advanced Image Classification with ImageNet


In this assignment, you will be asked to develop a convolutional neural network (CNN) to classify images from the CIFAR-100 dataset. At each step, you'll be guided through the process of developing a model architecture to solve a problem. Your goal is to create a CNN that attains at least 55% accuracy on the validation set.

### The CIFAR-100 Dataset

The [CIFAR-100 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) consists of 60000 32x32 colour images in 100 classes, with 600 images per class. There are 50000 training images and 10000 test images. The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 500 images from each class.

### Tools

You will use Keras with TensorFlow to develop your CNN. For this assignment, it's strongly recommended that you use a GPU to accelerate your training, or else you might find it difficult to train your network in a reasonable amount of time. If you have a computer with a GPU that you wish to use, you can follow the [TensorFlow instructions](https://www.tensorflow.org/install/) for installing TensorFlow with GPU support. Otherwise, you can use [Google Colab](https://colab.research.google.com/) to complete this assignment. Colab provides free access to GPU-enabled machines. If you run into any issues, please contact us as soon as possible so that we can help you resolve them.


## Task 1: Data Exploration and Preprocessing (Complete or Incomplete)

### 1a: Load and Explore the Dataset

- Use the code below to download the dataset.
- Explore the dataset: examine the shape of the training and test sets, the dimensions of the images, and the number of classes. Show a few examples from the training set.


In [ ]:
from keras.datasets import cifar100
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [ ]:
# Examine the shape of the training and test sets
print(f"Training data shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")

# Number of classes
num_classes = len(np.unique(y_train))
print(f"Number of classes: {num_classes}")

# Dimensions of the images
image_shape = x_train.shape[1:]
print(f"Image dimensions: {image_shape}")

# Display a few examples from the training set
def plot_examples(x, y, classes, num_examples=10):
    plt.figure(figsize=(15, 5))
    for i in range(num_examples):
        plt.subplot(2, 5, i + 1)
        plt.imshow(x[i])
        plt.title(f"Class: {y[i][0]}")
        plt.axis('off')
    plt.show()

# Plot 10 examples from the training set
plot_examples(x_train, y_train, num_classes)

### 1b: Data Preprocessing (4 Marks)

- With the data downloaded, it's time to preprocess it. Start by normalizing the images so that they all have pixel values in the range [0, 1].
- Next, convert the labels to one-hot encoded vectors.
- Finally, split the training set into training and validation sets. Use 80% of the training set for training and the remaining 20% for validation.


In [ ]:
# Normalize the images to have pixel values [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoded vectors
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Split the training set into training and validation sets (80% training, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Verify the shapes of the new training and validation sets
print(f"Training data shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation data shape: {x_val.shape}")
print(f"Validation labels shape: {y_val.shape}")

print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")

## Task 2: Model Development (Complete or Incomplete)

### Task 2a: Create a Baseline CNN Model

- Design a CNN architecture. Your architecture should use convolutional layers, max pooling layers, and dense layers. You can use any number of layers, and you can experiment with different numbers of filters, filter sizes, strides, padding, etc. The design doesn't need to be perfect, but it should be unique to you.
- Print out the model summary.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Create a baseline CNN model
model = Sequential()

# First convolutional block
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional block
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional block
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Fully connected layers
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.5))  # Add dropout for regularization
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.5))  # Add dropout for regularization

# Output layer
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print out the model summary
model.summary()


### Task 2b: Compile the model

- Select an appropriate loss function and optimizer for your model. These can be ones we have looked at already, or they can be different.


In [ ]:
from keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


- Briefly explain your choices (one or two sentences each).
- <b>Loss function:</b> I use categorical cross-entropy as our loss function because it works well for classifying images into 100 categories.
- <b>Optimizer:</b> I chose the Adam optimizer because it adjusts the learning rate to help the model converge faster and perform well for image classification tasks.


## Task 3: Model Training and Evaluation (Complete or Incomplete)

### Task 3a: Train the Model

- Train your model for an appropriate number of epochs. Explain your choice of the number of epochs used - you can change this number before submitting your assignment.

- I started with 20 epochs, a reasonable number to see if the model starts to overfit or if the validation accuracy continues to improve. We can adjust this number based on the model's performance.

- Use a batch size of 32.

- Is a common choice that balances memory usage and convergence speed.

- Use the validation set for validation.

- I used the validation set to monitor the model's performance and adjust hyperparameters as needed.


In [ ]:
# Number of epochs
epochs = 20
batch_size = 32

# Train the model
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")


### Task 3b: Accuracy and other relevant metrics on the test set

- Report the accuracy of your model on the test set.
- While accuracy is a good metric, there are many other ways to numerically evaluate a model. Report at least one other metric, and explain what it measures and how it is calculated.

- <b>Accuracy:</b> 0.36640000343322754

- <b>Other metrics:</b> precision, recall and F1-score.

- <b>Reason for selection:</b> The F1-score is selected because it provides a balance between precision and recall, especially useful when dealing with imbalanced datasets. It is the harmonic mean of precision and recall, giving a single metric that considers both false positives and false negatives.

- <b>Value of metric:</b> Precision: 0.3734036699660381, Recall: 0.3664 and 
F1-score: 0.3595827482163865

- <b>Interpretation of metric value:</b> A higher F1-score indicates better balance between precision and recall, meaning the model performs well in identifying positive instances without too many false positives or false negatives.


In [ ]:
from sklearn.metrics import classification_report

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

# Predict the labels for the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Generate a classification report
report = classification_report(y_true, y_pred_classes, output_dict=True)
print(classification_report(y_true, y_pred_classes))

# Extract the precision, recall, and F1-score for the overall performance
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1_score = report['weighted avg']['f1-score']

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1_score}")

### Task 3c: Visualize the model's learning

- Plot the training accuracy and validation accuracy with respect to epochs.

- Select an image that the model correctly classified in the test set, and an image that the model incorrectly classified in the test set. Plot the images and report the model's classification probabilities for each.

- Briefly discuss the results. What do the plots show? 
- The plots show the accuracy of the model on the training and validation sets over the epochs. If the validation accuracy is significantly lower than the training accuracy, it might indicate overfitting. If both accuracies improve and converge, it suggests good generalization.

Do the results make sense?
- For the Correctly Classified Image, the image is correctly classified by the model. The predicted probabilities should show a high value for the correct class, indicating the model's confidence.
- For the Incorrectly Classified Image, the image is incorrectly classified by the model. The predicted probabilities might be spread across different classes, indicating uncertainty, or show high confidence in an incorrect class, suggesting a model mistake.

- What do the classification probabilities indicate?
- The classification probabilities indicate the model's confidence in its predictions.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot training and validation accuracy
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

# Visualize the training history
plot_training_history(history)

# Select an image correctly classified by the model
correct_indices = np.where(y_pred_classes == y_true)[0]
incorrect_indices = np.where(y_pred_classes != y_true)[0]

def plot_image_with_probabilities(image, true_label, predicted_probs, classes):
    plt.imshow(image)
    plt.title(f"True label: {true_label}\nPredicted probabilities:\n{predicted_probs}")
    plt.axis('off')
    plt.show()

# Plot a correctly classified image
correct_index = correct_indices[0]
correct_image = x_test[correct_index]
correct_true_label = y_true[correct_index]
correct_predicted_probs = y_pred[correct_index]

plot_image_with_probabilities(correct_image, correct_true_label, correct_predicted_probs, num_classes)

# Plot an incorrectly classified image
incorrect_index = incorrect_indices[0]
incorrect_image = x_test[incorrect_index]
incorrect_true_label = y_true[incorrect_index]
incorrect_predicted_probs = y_pred[incorrect_index]

plot_image_with_probabilities(incorrect_image, incorrect_true_label, incorrect_predicted_probs, num_classes)

## Task 4: Model Enhancement (Complete or Incomplete)

### Task 4a: Implementation of at least one advanced technique

- Now it's time to improve your model. Implement at least one technique to improve your model's performance. You can use any of the techniques we have covered in class, or you can use a technique that we haven't covered. If you need inspiration, you can refer to the [Keras documentation](https://keras.io/).

**KERAS TECHNIQUE**: Data Augmentation

- Explain the technique you used and why you chose it.
Data augmentation increases the variability of the training dataset, making the model more robust and less likely to overfit. This can lead to improved performance on the validation and test sets.

- If you used a technique that requires tuning, explain how you selected the values for the hyperparameters.
I used a standard set of image adjustments without extensive fine-tuning. These adjustments typically work well for image classification tasks. However, I fine-tuned the choice of adjustments and their ranges based on experimentation.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

# Define the data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Fit the generator to the training data
datagen.fit(x_train)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Retrain the model using data augmentation
history_augmented = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // batch_size,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss_augmented, test_accuracy_augmented = model.evaluate(x_test, y_test)
print(f"Test loss after augmentation: {test_loss_augmented}")
print(f"Test accuracy after augmentation: {test_accuracy_augmented}")

# Generate predictions
y_pred_augmented = model.predict(x_test)
y_pred_classes_augmented = np.argmax(y_pred_augmented, axis=1)
y_true = np.argmax(y_test, axis=1)

# Generate a classification report
report_augmented = classification_report(y_true, y_pred_classes_augmented, output_dict=True)
print(classification_report(y_true, y_pred_classes_augmented))

# Extract the precision, recall, and F1-score for the overall performance
precision_augmented = report_augmented['weighted avg']['precision']
recall_augmented = report_augmented['weighted avg']['recall']
f1_score_augmented = report_augmented['weighted avg']['f1-score']

print(f"Precision after augmentation: {precision_augmented}")
print(f"Recall after augmentation: {recall_augmented}")
print(f"F1-score after augmentation: {f1_score_augmented}")

### Task 4b: Evaluation of the enhanced model

- Re-train your model using the same number of epochs as before.
- Compare the accuracy and other selected metric on the test set to the results you obtained before.
- As before, plot the training accuracy and validation accuracy with respect to epochs, and select an image that the model correctly classified in the test set, and an image that the model incorrectly classified in the test set. Plot the images and report the model's classification probabilities for each.


In [ ]:
def plot_training_history(history, title=None):
    # Your code here
    if title:
        plt.title(title)
    # More of your code here


# Plot training and validation accuracy
plot_training_history(history_augmented, title='Training and Validation Accuracy with Data Augmentation')

# Select an image correctly classified by the model
correct_indices_augmented = np.where(y_pred_classes_augmented == y_true)[0]
incorrect_indices_augmented = np.where(y_pred_classes_augmented != y_true)[0]

# Plot a correctly classified image
correct_index_augmented = correct_indices_augmented[0]
correct_image_augmented = x_test[correct_index_augmented]
correct_true_label_augmented = y_true[correct_index_augmented]
correct_predicted_probs_augmented = y_pred_augmented[correct_index_augmented]

plot_image_with_probabilities(correct_image_augmented, correct_true_label_augmented, correct_predicted_probs_augmented, num_classes)

# Plot an incorrectly classified image
incorrect_index_augmented = incorrect_indices_augmented[0]
incorrect_image_augmented = x_test[incorrect_index_augmented]
incorrect_true_label_augmented = y_true[incorrect_index_augmented]
incorrect_predicted_probs_augmented = y_pred_augmented[incorrect_index_augmented]

plot_image_with_probabilities(incorrect_image_augmented, incorrect_true_label_augmented, incorrect_predicted_probs_augmented, num_classes)


### Task 4c: Discussion of the results

- Briefly discuss the results.
- Did the model's performance improve?
- Yes, the model's performance improved. The test accuracy increased from 36.64% to 40.77%, and there were improvements in precision, recall, and F1-score as well.

- Why do you think this is?
- The improvement is likely due to data augmentation. By artificially increasing the diversity of the training data through transformations like rotations, shifts, flips, and zooms, the model learned to generalize better and became more robust to variations in the images, reducing overfitting.

- Do you think there is room for further improvement? Why or why not?
- there is room for further improvement. The current performance indicates that the model can still be enhanced. Techniques such as more sophisticated data augmentation, regularization methods like dropout and batch normalization, and fine-tuning the model architecture and hyperparameters could further boost performance.

- What other techniques might you try in the future?
- Dropout: To prevent overfitting by randomly dropping units during training.
- Learning Rate Schedulers: To adjust the learning rate dynamically.
- Transfer Learning: Using pre-trained models and fine-tuning them on CIFAR-100.

- Your answer should be no more than 200 words.

## Criteria

| Criteria | Complete                                                          | Incomplete                                                    |
| -------- | ----------------------------------------------------------------- | ------------------------------------------------------------- |
| Task 1   | The task has been completed successfully and there are no errors. | The task is still incomplete and there is at least one error. |
| Task 2   | The task has been completed successfully and there are no errors. | The task is still incomplete and there is at least one error. |
| Task 3   | The task has been completed successfully and there are no errors. | The task is still incomplete and there is at least one error. |
| Task 4   | The task has been completed successfully and there are no errors. | The task is still incomplete and there is at least one error. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:

- Submission Due Date: `11:59 PM - 28/07/2024`
- The branch name for your repo should be: `assignment-1`
- What to submit for this assignment:
  - This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/fredylrincon/deep_learning/pull/<pr_id>`
  - Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:

- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
